In [41]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon

import warnings
warnings.filterwarnings('ignore')

In [37]:
class AlgorithmComparisonData():
    def __init__(self, wins, significant_wins, defeats, significant_defeats):
        self.wins = wins
        self.significant_wins = significant_wins
        self.defeats = defeats
        self.significant_defeats = significant_defeats

    def __str__(self):
        return f"wins: {self.wins}({self.significant_wins})\ndefeats: {self.defeats}({self.significant_defeats})"

class ExperimentsDataAnalyzer():

    def __init__(self):
        self.data_lake = {}

    def load_data(self, data_source_name, data_source_path):
        df = pd.read_csv(data_source_path, sep=';')
        self.data_lake[data_source_name] = df

    def get_data_frame(self, data_source_name):
        return self.data_lake[data_source_name]

    def compare(self, ref_alg_datasource_name, target_alg_datasource_name):
        ref_df = self.get_data_frame(ref_alg_datasource_name)
        tgt_df = self.get_data_frame(target_alg_datasource_name)
        
        wins, swins = 0, 0 
        defeats, sdefeats = 0, 0
    
        for i in range(len(tgt_df)):
            data1 = ref_df.iloc[i][1:]
            data2 = tgt_df.iloc[i][1:]
            
            d1_mean = np.mean(data1)
            d2_mean = np.mean(data2) 
        
            is_signif = False
            if d1_mean == 0.0 and d2_mean == 0.0:
                is_signif = False
            else:
                stat, p = wilcoxon(data1, data2, zero_method="zsplit", )
                if p <= 0.05:
                    is_signif = True
            
            if d1_mean > d2_mean:
                wins += 1
                if is_signif:
                    swins += 1
        
            elif d1_mean < d2_mean:
                defeats += 1
                if is_signif:
                    sdefeats += 1

        return AlgorithmComparisonData(wins, swins, defeats, sdefeats)

In [38]:
analyzer = ExperimentsDataAnalyzer()

base_path = "/home/raphaelgs/code/FIM-CGRASP/results/stats_test_data"

alg_ref_ds_name = "lshade_costs_by_seed"
alg_ref_path = f"{base_path}/out(--lshade ).csv"

alg_target_ds_name = "dmlshade_costs_by_seed"
alg_target_path = f"{base_path}/uv=10/out(--dmlshade --crossgen --insert --k 18 --elsz 170 --dms 57 ).csv"

analyzer.load_data(alg_ref_ds_name, alg_ref_path)
analyzer.load_data(alg_target_ds_name, alg_target_path)

In [43]:
analyzer.get_data_frame(alg_ref_ds_name).head()

,f,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30
0,1-10,0.0000,0.00000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.00000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000
1,2-10,0.0000,0.00000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.00000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000
2,3-10,0.0000,0.00000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.00000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000
3,4-10,34.7803,34.78030,34.780300,34.7803,34.7803,34.7803,34.7803,34.7803,34.7803,...,34.7803,34.78030,34.7803,34.780300,34.7803,34.7803,34.7803,34.7803,34.7803,4.33541
4,5-10,20.0023,2.38704,0.212092,20.0052,19.9999,20.0060,20.0119,20.0078,20.0111,...,19.9997,2.69013,19.8769,0.123925,20.0048,20.0143,20.0135,20.0132,20.0051,20.00610


In [46]:
results = analyzer.compare(alg_ref_ds_name, alg_target_ds_name)
print(results)

wins: 17(9)
defeats: 6(0)
